In [1]:
import numpy as np
import re
import functools

with open("input.txt") as f:
    p = r"((?:on)|(?:off)) x=(-?\d*?)\.\.(\d*?)(-?\d*?),y=(-?\d*?)\.\.(\d*?)(-?\d*?),z=(-?\d*?)\.\.(\d*?)(-?\d*)"
    data = [re.match(p,x) for x in f.read().split('\n')]
    data=np.array([
        (int(x.group(1)=='on'),                          #+1 if on, -1 if off
         int(x.group(2)), int(x.group(5)), int(x.group(8 )), #bottom-left point
         int(x.group(4)), int(x.group(7)), int(x.group(10))) #top-right point
         for x in data], dtype=np.int64)
data

array([[     1,    -29,    -30, ...,     23,     15,     49],
       [     1,    -15,    -36, ...,     38,     10,      7],
       [     1,    -35,    -28, ...,     17,     24,      2],
       ...,
       [     1,  44297,  12992, ...,  63319,  24574,  65139],
       [     0,  20452, -73174, ...,  22744, -56753, -21933],
       [     1, -50494,  67270, ..., -27967,  81084,  -1163]], dtype=int64)

In [4]:
class prism:
    def __init__(self, v):
        self.v = v
        self.p1 = v[1:4]
        self.p2 = v[4:]
        self.val, self.x1, self.y1, self.z1, self.x2, self.y2, self.z2 = v
        self.dx, self.dy, self.dz = np.abs(self.p1-self.p2) + 1
        self.size = self.dx*self.dy*self.dz
    
    @functools.lru_cache(maxsize=0) #this doesnt actually do anything
    def intersection_area(self, other):
        ov1 = np.max(np.vstack((self.p1, other.p1)), axis=0)
        ov2 = np.min(np.vstack((self.p2, other.p2)), axis=0)
        if (ov1 > ov2).any():
            return False, False, False
        return np.prod(np.abs(ov2 - ov1) + 1), ov1, ov2
    

prisms = []
count = 0
pt1 = True
for row in data:
    if pt1 and (np.abs(row) > 50).any():
        print(f"part 1: {count}")
        pt1 = False
    new = prism(row)
    if new.val == 1:
        count += new.size
    intersections = []
    for old in prisms:
        _count, ov1, ov2 = new.intersection_area(old)
        if _count:
            count -= _count*old.val
            intersections.append(prism(np.hstack((-old.val, ov1, ov2)))) #not super happy with the runtime but meh
    if new.val != 0:
        prisms.append(new)
    prisms += intersections
    #print(count)
print(f"part 2: {count}")

part 1: 537042
part 2: 1304385553084863
